<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 36px">Notebook de exemplo para como corretamente fazer o ETL dos dados - MS</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 22px; padding: 5px; border-radius: 5px;">
  <li>Apresenta apenas como organizar os dados com um exemplo em JSON.</li>
  <li>Quais devem ser os atributos para envio no Postgre DB.</li>
</ul>

In [11]:
import pandas as pd
import numpy as np
import os, json
import itertools

<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 22px;">Importando dados</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 16px; padding: 5px; border-radius: 5px;">
  <li>Como coletar os dados das interações via email</li>
  <li>O exemplo executado abaixo, é muito simples e apenas um exemplo dos regras de negócio que devem ser implementadas, atributos e o JSON podem estar diferentes da realidade</li>
</ul>

In [ ]:
# Importando os dados de exemplo do Mail API
# Obviamente esses dados serão extraídos diretamente da chamada da API, aqui é apenas um exemplo.

with open('msmail_data_example.json', 'r') as file:
    # Coletando os dados do arquivo local
    data = json.load(file)
    result_mail = []

    # Coletando os dados dos emails extraídos.
    # Provavelmente na extração que você fará, cada user terá uma série de JSON's, que você deve iterar e coletar os items
    for mail in data['emails']:
        # Aqui nos coletamos quem foi o individuo que enviou o email
        sender = [mail['from']['emailAddress']['address']]
        # Aqui coletamos apenas quem recebeu o email, não leve em consideração os CC e BCC
        to_receivers = [person['emailAddress']['address'] for person in mail['toRecipients']]

        # Aqui nos fazemos um append de ambas as listas para fazer a combinação 2 a 2 de cada participante que estava no email
        # Pode existir combinações remetente > destinatário como destinatário > destinatário. É importante existir ambas as relações
        participantes = sender + to_receivers
        combinations = list(itertools.combinations(participantes, 2))    

        # Aqui criamos um Dict que será utilizados para criar um DataFrame posteriormente
        # internetMessageId é o identificador único do email entre todas as caixas de email. NÃO UTILIZE O IDENTIFICADOR "ID"
        result_mail.extend([
            {
                'internetMessageId': mail['internetMessageId'],
                'sender':dupla[0],
                'receiver':dupla[1],
                'data': pd.Timestamp(mail['createdDateTime']).date(),
                'de_onde':'mail'
            }
            for dupla in combinations
        ])          

result_mail = pd.DataFrame(result_mail) ; result_mail

,internetMessageId,sender,receiver,data,de_onde
0,<projupdate123@company.com>,bob.smith@company.com,alice.johnson@company.com,2025-03-01,mail
1,<projupdate123@company.com>,bob.smith@company.com,super.john@company.com,2025-03-01,mail
2,<projupdate123@company.com>,alice.johnson@company.com,super.john@company.com,2025-03-01,mail
3,<finreportQ1@company.com>,david.lee@company.com,alice.johnson@company.com,2025-03-02,mail
4,<finreportQ1@company.com>,david.lee@company.com,alice.johnson@company.com,2025-03-02,mail
5,<vlanhex@company.com>,victor.lav@company.com,alice.johnson@company.com,2025-03-02,mail


<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 22px;">Caso de agrupamento</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 16px; padding: 5px; border-radius: 5px;">
  <li>Perceba como acima, temos duas vezes o finreportQ1 no internetMessageId aparecendo para exatamente sender e receiver iguais.</li>
  <li>Isso acontece pois extraímos os dados de duas caixas de email diferentes e por isso precisamos eliminar essa duplicação para isso criamos um hash_id que mapeia a relação</li>
</ul>

In [ ]:
# Utilize dessa função para criar o hashmap dos colaboradores da organização
# Queremos mapear a relação entre 2 colaboradores independente da ordem das colunas
import hashlib
def generate_hash(sender, receiver):
    interaction = f"{min(sender, receiver)}-{max(sender, receiver)}"
    return hashlib.sha256(interaction.encode()).hexdigest()

result_mail['hash_id'] = result_mail.apply(lambda row: generate_hash(row['sender'], row['receiver']), axis=1) ; result_mail

,internetMessageId,sender,receiver,data,de_onde,hash_id
0,<projupdate123@company.com>,bob.smith@company.com,alice.johnson@company.com,2025-03-01,mail,43cc8c9abcb9beca5d3fa06e0645ec9cbe4802b6d46e40...
1,<projupdate123@company.com>,bob.smith@company.com,super.john@company.com,2025-03-01,mail,20fdfe232f445756a52d113f3347e61d5426271d6c2c55...
2,<projupdate123@company.com>,alice.johnson@company.com,super.john@company.com,2025-03-01,mail,38ac789e480a65b9ac0ed2d286485c45e96bc12032267c...
3,<finreportQ1@company.com>,david.lee@company.com,alice.johnson@company.com,2025-03-02,mail,2eff1241584d53d95c9a8642ada2d9a2d2d7586a3477b8...
4,<finreportQ1@company.com>,david.lee@company.com,alice.johnson@company.com,2025-03-02,mail,2eff1241584d53d95c9a8642ada2d9a2d2d7586a3477b8...
5,<vlanhex@company.com>,victor.lav@company.com,alice.johnson@company.com,2025-03-02,mail,e69a9c3a8a4b417c3a6ff84650d4c3970698417cf7a388...


In [14]:
result_mail[result_mail['internetMessageId'] == '<finreportQ1@company.com>']['hash_id'].value_counts()

2eff1241584d53d95c9a8642ada2d9a2d2d7586a3477b873112c39b7cb887aee    2
Name: hash_id, dtype: int64

<p style="text-align: center; color: #D6A4FF; font-family: 'Montserrat'; font-weight: bold; font-size: 22px;">Caso de agrupamento</p>

<ul style="font-family: 'Montserrat'; color: white; background-color: #FFFFF; font-size: 16px; padding: 5px; border-radius: 5px;">
  <li>Perceba como acima, como realmente para um internetMessageId temos 2 hash_ids iguais, precisamos agrupar e eliminar duplicatas</li>
</ul>

In [15]:
result_mail.groupby(['internetMessageId','hash_id']).agg(
    sender=('sender','max'),
    receiver=('receiver','max'),
    data=('data','max'),
    de_onde=('de_onde','max')
).reset_index()

,internetMessageId,hash_id,sender,receiver,data,de_onde
0,<finreportQ1@company.com>,2eff1241584d53d95c9a8642ada2d9a2d2d7586a3477b8...,david.lee@company.com,alice.johnson@company.com,2025-03-02,mail
1,<projupdate123@company.com>,20fdfe232f445756a52d113f3347e61d5426271d6c2c55...,bob.smith@company.com,super.john@company.com,2025-03-01,mail
2,<projupdate123@company.com>,38ac789e480a65b9ac0ed2d286485c45e96bc12032267c...,alice.johnson@company.com,super.john@company.com,2025-03-01,mail
3,<projupdate123@company.com>,43cc8c9abcb9beca5d3fa06e0645ec9cbe4802b6d46e40...,bob.smith@company.com,alice.johnson@company.com,2025-03-01,mail
4,<vlanhex@company.com>,e69a9c3a8a4b417c3a6ff84650d4c3970698417cf7a388...,victor.lav@company.com,alice.johnson@company.com,2025-03-02,mail


### Critérios de aceite:

* id não é o identificador único entre todas as agendas, o internetMessageId que realiza esse papel.
* É possível existir internetMessageId duplicados.
* É possível existir hash_id duplicados.
* Não se pode possuir internetMessageId e hash_id duplicados.
* É necessário realizar as combinações par a par de todos os participantes do From e do to_recipients da reunião utilizando o itertools como foi demonstrado.